# 1. Import and Install Dependencies

In [4]:
# !pip install tensorflow==2.5.0 tensorflow-gpu==2.5.0 opencv-python mediapipe sklearn matplotlib

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021.2.3, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
bokeh 2.4.1 requires typing-extensions>=3.10.0, but you have typing-extensions 3.7.4.3 which is incompatible.



  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=f03663cc621b0a53dab3b69dc3d32585f5c9c7bb387c33f3c0f62992de746668
  Stored in directory: c:\users\yours\appdata\local\pip\cache\wheels\e4\7b\98\b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio-1.42.0:
      Successfully uninstalled grpcio-1.42.0
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.0.0
    Uninstalling absl-py-1.0.0:
      Successfully uninstalled absl-py-1.0.0
  Attempting uninstall: typing

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Hand Detection using Mediapipe Holistic model

In [2]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False                 
    results = model.process(image)                
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_styled_landmarks(image, results):

    # left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [5]:
def extract_keypoints(results):
    key1 = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    key2 = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([key1, key2, lh, rh])

# 4. Setup Folders for Collection

In [6]:

DATA_PATH = os.path.join('dynamic_data') 


actions = np.array(['hello', 'thanks', 'i love you', 'stop', 'please', 'walk', 'argue', 'yes', 'see', 'good'])

# 30 videos
no_sequences = 40

# 30 frames
sequence_length = 30

In [7]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [8]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  
    for action in actions:
       
        for sequence in range(no_sequences):
           
            for frame_num in range(sequence_length):

              
                ret, frame = cap.read()

                image, results = mediapipe_detection(frame, holistic)


                draw_styled_landmarks(image, results)
                

                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                  
                    cv2.imshow('OpenCV Feed', image)
                
              
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)


                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [9]:
# cap.release()
# cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels 

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
label_map

{'hello': 0,
 'thanks': 1,
 'i love you': 2,
 'stop': 3,
 'please': 4,
 'walk': 5,
 'argue': 6,
 'yes': 7,
 'see': 8,
 'good': 9}

In [13]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [14]:
X = np.array(sequences)

In [15]:
y = to_categorical(labels).astype(int)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [17]:
y_test.shape

(20, 10)

In [18]:
X_train.shape

(380, 30, 1662)

# 7. Build and Train LSTM Neural Network

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard

In [20]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [21]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))


model.add(LSTM(128, return_sequences=True, activation='relu'))


model.add(LSTM(64, return_sequences=False, activation='relu'))


model.add(Dense(64, activation='relu'))


model.add(Dense(32, activation='relu'))


model.add(Dense(actions.shape[0], activation='softmax'))

In [22]:
model.compile(optimizer= 'Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [23]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback], validation_data = (X_test, y_test))

Epoch 1/500
12/12 [==============================] - 5s 136ms/step - loss: 2.8554 - categorical_accuracy: 0.0816 - val_loss: 2.2776 - val_categorical_accuracy: 0.0000e+00
Epoch 2/500
12/12 [==============================] - 1s 69ms/step - loss: 2.3059 - categorical_accuracy: 0.0868 - val_loss: 2.2545 - val_categorical_accuracy: 0.0500
Epoch 3/500
12/12 [==============================] - 1s 69ms/step - loss: 2.2374 - categorical_accuracy: 0.1763 - val_loss: 2.0274 - val_categorical_accuracy: 0.3500
Epoch 4/500
12/12 [==============================] - 1s 60ms/step - loss: 2.2226 - categorical_accuracy: 0.1474 - val_loss: 2.2397 - val_categorical_accuracy: 0.1000
Epoch 5/500
12/12 [==============================] - 1s 62ms/step - loss: 2.1201 - categorical_accuracy: 0.1711 - val_loss: 2.1283 - val_categorical_accuracy: 0.1500
Epoch 6/500
12/12 [==============================] - 1s 64ms/step - loss: 2.0251 - categorical_accuracy: 0.1974 - val_loss: 1.9921 - val_categorical_accuracy: 0.2000

Epoch 50/500
12/12 [==============================] - 1s 64ms/step - loss: 2.2875 - categorical_accuracy: 0.1184 - val_loss: 2.2617 - val_categorical_accuracy: 0.2500
Epoch 51/500
12/12 [==============================] - 1s 61ms/step - loss: 2.3047 - categorical_accuracy: 0.0921 - val_loss: 2.3188 - val_categorical_accuracy: 0.0500
Epoch 52/500
12/12 [==============================] - 1s 59ms/step - loss: 2.3026 - categorical_accuracy: 0.1026 - val_loss: 2.3210 - val_categorical_accuracy: 0.0500
Epoch 53/500
12/12 [==============================] - 1s 61ms/step - loss: 2.3028 - categorical_accuracy: 0.1026 - val_loss: 2.3208 - val_categorical_accuracy: 0.0500
Epoch 54/500
12/12 [==============================] - 1s 62ms/step - loss: 2.3027 - categorical_accuracy: 0.1026 - val_loss: 2.3199 - val_categorical_accuracy: 0.0500
Epoch 55/500
12/12 [==============================] - 1s 65ms/step - loss: 2.3028 - categorical_accuracy: 0.1026 - val_loss: 2.3185 - val_categorical_accuracy: 0.050

Epoch 99/500
12/12 [==============================] - 1s 62ms/step - loss: 1.7266 - categorical_accuracy: 0.2684 - val_loss: 1.5428 - val_categorical_accuracy: 0.4500
Epoch 100/500
12/12 [==============================] - 1s 58ms/step - loss: 1.6345 - categorical_accuracy: 0.3000 - val_loss: 2.2273 - val_categorical_accuracy: 0.1500
Epoch 101/500
12/12 [==============================] - 1s 67ms/step - loss: 1.8060 - categorical_accuracy: 0.2684 - val_loss: 1.5596 - val_categorical_accuracy: 0.4000
Epoch 102/500
12/12 [==============================] - 1s 66ms/step - loss: 1.6878 - categorical_accuracy: 0.3342 - val_loss: 1.7764 - val_categorical_accuracy: 0.3500
Epoch 103/500
12/12 [==============================] - 1s 62ms/step - loss: 2.0876 - categorical_accuracy: 0.2500 - val_loss: 2.8228 - val_categorical_accuracy: 0.1000
Epoch 104/500
12/12 [==============================] - 1s 60ms/step - loss: 2.3928 - categorical_accuracy: 0.1026 - val_loss: 2.3634 - val_categorical_accuracy: 

12/12 [==============================] - 1s 62ms/step - loss: 0.9906 - categorical_accuracy: 0.5500 - val_loss: 1.2438 - val_categorical_accuracy: 0.5500
Epoch 148/500
12/12 [==============================] - 1s 60ms/step - loss: 1.0631 - categorical_accuracy: 0.5263 - val_loss: 1.2751 - val_categorical_accuracy: 0.5000
Epoch 149/500
12/12 [==============================] - 1s 60ms/step - loss: 1.0986 - categorical_accuracy: 0.5368 - val_loss: 1.4614 - val_categorical_accuracy: 0.4000
Epoch 150/500
12/12 [==============================] - 1s 59ms/step - loss: 1.0470 - categorical_accuracy: 0.5553 - val_loss: 1.2266 - val_categorical_accuracy: 0.6500
Epoch 151/500
12/12 [==============================] - 1s 62ms/step - loss: 0.9179 - categorical_accuracy: 0.6184 - val_loss: 1.2314 - val_categorical_accuracy: 0.8500
Epoch 152/500
12/12 [==============================] - 1s 63ms/step - loss: 0.9540 - categorical_accuracy: 0.6184 - val_loss: 1.0724 - val_categorical_accuracy: 0.7500
Epoch 

12/12 [==============================] - 1s 64ms/step - loss: 0.4961 - categorical_accuracy: 0.8105 - val_loss: 1.0057 - val_categorical_accuracy: 0.5500
Epoch 196/500
12/12 [==============================] - 1s 62ms/step - loss: 0.4635 - categorical_accuracy: 0.8184 - val_loss: 0.7763 - val_categorical_accuracy: 0.7500
Epoch 197/500
12/12 [==============================] - 1s 65ms/step - loss: 0.5019 - categorical_accuracy: 0.8132 - val_loss: 0.9538 - val_categorical_accuracy: 0.6000
Epoch 198/500
12/12 [==============================] - 1s 65ms/step - loss: 0.5009 - categorical_accuracy: 0.7816 - val_loss: 1.0049 - val_categorical_accuracy: 0.6000
Epoch 199/500
12/12 [==============================] - 1s 65ms/step - loss: 0.5179 - categorical_accuracy: 0.7658 - val_loss: 1.0508 - val_categorical_accuracy: 0.6000
Epoch 200/500
12/12 [==============================] - 1s 67ms/step - loss: 0.5148 - categorical_accuracy: 0.7789 - val_loss: 0.7947 - val_categorical_accuracy: 0.7500
Epoch 

...

Epoch 293/500
12/12 [==============================] - 1s 64ms/step - loss: 0.3792 - categorical_accuracy: 0.8263 - val_loss: 0.4800 - val_categorical_accuracy: 0.8000
Epoch 294/500
12/12 [==============================] - 1s 62ms/step - loss: 0.3072 - categorical_accuracy: 0.8421 - val_loss: 0.6330 - val_categorical_accuracy: 0.6000
Epoch 295/500
12/12 [==============================] - 1s 64ms/step - loss: 0.2405 - categorical_accuracy: 0.8947 - val_loss: 0.4254 - val_categorical_accuracy: 0.8000
Epoch 296/500
12/12 [==============================] - 1s 61ms/step - loss: 0.2099 - categorical_accuracy: 0.9158 - val_loss: 0.4865 - val_categorical_accuracy: 0.7500
Epoch 297/500
12/12 [==============================] - 1s 64ms/step - loss: 0.2077 - categorical_accuracy: 0.9000 - val_loss: 0.6154 - val_categorical_accuracy: 0.6500
Epoch 298/500
12/12 [==============================] - 1s 65ms/step - loss: 0.4437 - categorical_accuracy: 0.8132 - val_loss: 1.5071 - val_categorical_accuracy:

Epoch 342/500
12/12 [==============================] - 1s 65ms/step - loss: 0.1442 - categorical_accuracy: 0.9447 - val_loss: 1.1079 - val_categorical_accuracy: 0.6000
Epoch 343/500
12/12 [==============================] - 1s 67ms/step - loss: 0.1713 - categorical_accuracy: 0.9316 - val_loss: 0.3612 - val_categorical_accuracy: 0.9000
Epoch 344/500
12/12 [==============================] - 1s 63ms/step - loss: 0.1913 - categorical_accuracy: 0.9316 - val_loss: 0.5163 - val_categorical_accuracy: 0.8000
Epoch 345/500
12/12 [==============================] - 1s 64ms/step - loss: 0.1534 - categorical_accuracy: 0.9342 - val_loss: 0.3280 - val_categorical_accuracy: 0.8500
Epoch 346/500
12/12 [==============================] - 1s 65ms/step - loss: 0.2251 - categorical_accuracy: 0.9211 - val_loss: 0.3700 - val_categorical_accuracy: 0.8500
Epoch 347/500
12/12 [==============================] - 1s 65ms/step - loss: 0.2908 - categorical_accuracy: 0.8868 - val_loss: 0.4977 - val_categorical_accuracy:

Epoch 391/500
12/12 [==============================] - 1s 61ms/step - loss: 0.3929 - categorical_accuracy: 0.8474 - val_loss: 0.4347 - val_categorical_accuracy: 0.8500
Epoch 392/500
12/12 [==============================] - 1s 62ms/step - loss: 0.2541 - categorical_accuracy: 0.9079 - val_loss: 0.4181 - val_categorical_accuracy: 0.7500
Epoch 393/500
12/12 [==============================] - 1s 62ms/step - loss: 0.1977 - categorical_accuracy: 0.9211 - val_loss: 0.4887 - val_categorical_accuracy: 0.7500
Epoch 394/500
12/12 [==============================] - 1s 61ms/step - loss: 0.1683 - categorical_accuracy: 0.9316 - val_loss: 0.3766 - val_categorical_accuracy: 0.7500
Epoch 395/500
12/12 [==============================] - 1s 64ms/step - loss: 0.1225 - categorical_accuracy: 0.9605 - val_loss: 0.3550 - val_categorical_accuracy: 0.8500
Epoch 396/500
12/12 [==============================] - 1s 64ms/step - loss: 0.0951 - categorical_accuracy: 0.9658 - val_loss: 0.5458 - val_categorical_accuracy:

Epoch 440/500
12/12 [==============================] - 1s 67ms/step - loss: 0.0319 - categorical_accuracy: 0.9921 - val_loss: 0.6856 - val_categorical_accuracy: 0.8500
Epoch 441/500
12/12 [==============================] - 1s 61ms/step - loss: 0.0268 - categorical_accuracy: 0.9921 - val_loss: 0.6793 - val_categorical_accuracy: 0.8000
Epoch 442/500
12/12 [==============================] - 1s 62ms/step - loss: 0.0317 - categorical_accuracy: 0.9947 - val_loss: 0.8478 - val_categorical_accuracy: 0.8500
Epoch 443/500
12/12 [==============================] - 1s 63ms/step - loss: 0.0285 - categorical_accuracy: 0.9921 - val_loss: 0.8015 - val_categorical_accuracy: 0.8000
Epoch 444/500
12/12 [==============================] - 1s 63ms/step - loss: 0.0437 - categorical_accuracy: 0.9842 - val_loss: 1.0075 - val_categorical_accuracy: 0.6500
Epoch 445/500
12/12 [==============================] - 1s 60ms/step - loss: 0.0908 - categorical_accuracy: 0.9632 - val_loss: 0.8416 - val_categorical_accuracy:

Epoch 489/500
12/12 [==============================] - 1s 63ms/step - loss: 0.0012 - categorical_accuracy: 1.0000 - val_loss: 0.8706 - val_categorical_accuracy: 0.9000
Epoch 490/500
12/12 [==============================] - 1s 63ms/step - loss: 0.0012 - categorical_accuracy: 1.0000 - val_loss: 0.8659 - val_categorical_accuracy: 0.9000
Epoch 491/500
12/12 [==============================] - 1s 65ms/step - loss: 0.0011 - categorical_accuracy: 1.0000 - val_loss: 0.8721 - val_categorical_accuracy: 0.9000
Epoch 492/500
12/12 [==============================] - 1s 66ms/step - loss: 0.0010 - categorical_accuracy: 1.0000 - val_loss: 0.8655 - val_categorical_accuracy: 0.9500
Epoch 493/500
12/12 [==============================] - 1s 67ms/step - loss: 0.0010 - categorical_accuracy: 1.0000 - val_loss: 0.8805 - val_categorical_accuracy: 0.9000
Epoch 494/500
12/12 [==============================] - 1s 69ms/step - loss: 9.5590e-04 - categorical_accuracy: 1.0000 - val_loss: 0.8878 - val_categorical_accur

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 596,906
Trainable params: 596,906
Non-trainable params: 0
__________________________________________________

# 8. Make Predictions

In [25]:
res = model.predict(X_test)

In [26]:
actions[np.argmax(res[4])]

'thanks'

In [27]:
actions[np.argmax(y_test[4])]

'thanks'

# 9. Save Weights

In [28]:
from tensorflow import keras

In [29]:
 model.save('recognition.h5')

In [30]:
modelF= keras.models.load_model('recognition.h5')

In [31]:
model.load_weights('recognition.h5')

# 10. Evaluation and Accuracy

In [32]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [41]:
yhat = modelF.predict(X_test)

In [42]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [43]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[18,  0],
        [ 0,  2]],

       [[17,  0],
        [ 0,  3]],

       [[17,  0],
        [ 2,  1]],

       [[19,  0],
        [ 0,  1]],

       [[19,  0],
        [ 0,  1]],

       [[18,  0],
        [ 0,  2]],

       [[15,  1],
        [ 0,  4]],

       [[17,  1],
        [ 0,  2]],

       [[18,  0],
        [ 0,  2]]], dtype=int64)

In [44]:
accuracy_score(ytrue, yhat)

0.9

# 11. Test in Real Time

In [45]:
colors = [(245,117,16), (117,245,16), (16,117,245),(16,17,245),(16,117,24),(17,25,160),(11,45,116),(170,205,165), (224, 32, 28), (22,142,100)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [49]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.4

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)

        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = modelF.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('Recognition Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
walk
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
walk
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
walk
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<cla

<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
yes
<class 'mediapipe.python.solution_base.SolutionOutputs'>


<class 'mediapipe.python.solution_base.SolutionOutputs'>
stop
<class 'mediapipe.python.solution_base.SolutionOutputs'>
stop
<class 'mediapipe.python.solution_base.SolutionOutputs'>
stop
<class 'mediapipe.python.solution_base.SolutionOutputs'>
stop
<class 'mediapipe.python.solution_base.SolutionOutputs'>
stop
<class 'mediapipe.python.solution_base.SolutionOutputs'>
stop
<class 'mediapipe.python.solution_base.SolutionOutputs'>
argue
<class 'mediapipe.python.solution_base.SolutionOutputs'>
argue
<class 'mediapipe.python.solution_base.SolutionOutputs'>
argue
<class 'mediapipe.python.solution_base.SolutionOutputs'>
argue
<class 'mediapipe.python.solution_base.SolutionOutputs'>
stop
<class 'mediapipe.python.solution_base.SolutionOutputs'>
argue
<class 'mediapipe.python.solution_base.SolutionOutputs'>
argue
<class 'mediapipe.python.solution_base.SolutionOutputs'>
argue
<class 'mediapipe.python.solution_base.SolutionOutputs'>
argue
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good


<class 'mediapipe.python.solution_base.SolutionOutputs'>
see
<class 'mediapipe.python.solution_base.SolutionOutputs'>
see
<class 'mediapipe.python.solution_base.SolutionOutputs'>
see
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
yes
<class 'mediapipe.python.solution_base.SolutionOutputs'>
yes
<class 'mediapipe.python.solution_base.SolutionOutputs'>
yes
<class 'me

<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'

<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'